In [ ]:
import os
os.environ['WANDB_DISABLED']='true'

In [42]:
!pip install datasets

In [43]:
import pandas as pd
from datasets import Dataset

In [44]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# dataset = load_dataset("cnn_dailymail", "1.0.0")
train_df = pd.read_csv('/content/drive/MyDrive/Project/train_data.csv')
validation_df = pd.read_csv('/content/drive/MyDrive/Project/validation_data.csv')

In [47]:
test_df = pd.read_csv('/content/drive/MyDrive/Project/test_data.csv')

In [48]:
train_subset = train_df.sample(n=40000,random_state=42)
val_subset = validation_df.sample(n=10000,random_state=42)

In [ ]:
# Load tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_subset)
validation_dataset = Dataset.from_pandas(val_subset)

In [ ]:
# Prepare the data
def preprocess_data(data, tokenizer, max_input_length=512, max_target_length=150):
    inputs = data['article']
    targets = data['highlights']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(lambda x: preprocess_data(x, tokenizer), batched=True)
validation_dataset = validation_dataset.map(lambda x: preprocess_data(x, tokenizer), batched=True)


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
columns = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type="torch", columns=columns)
validation_dataset.set_format(type="torch", columns=columns)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Project/t5-small-summarizer",
    eval_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_dir="/content/drive/MyDrive/Project/logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

<ipython-input-45-a646671eeb96>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
100,0.955700,1.122964
200,0.909500,1.118278
300,0.927600,1.127757
400,0.925500,1.109588
500,1.000100,1.091814
600,0.978500,1.089461
700,0.979800,1.079751
800,0.966900,1.079056
900,0.974900,1.078715
1000,0.953200,1.078944


Step,Training Loss,Validation Loss
100,0.955700,1.122964
200,0.909500,1.118278
300,0.927600,1.127757
400,0.925500,1.109588
500,1.000100,1.091814
600,0.978500,1.089461
700,0.979800,1.079751
800,0.966900,1.079056
900,0.974900,1.078715
1000,0.953200,1.078944


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5000, training_loss=0.9497454040527343, metrics={'train_runtime': 9491.1829, 'train_samples_per_second': 4.214, 'train_steps_per_second': 0.527, 'total_flos': 5413672058880000.0, 'train_loss': 0.9497454040527343, 'epoch': 1.0})

In [49]:
model_name='/content/drive/MyDrive/Project/t5-small-summarizer/checkpoint-5000'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [50]:
# sample_text = test_df.iloc[1]['article']
sample_text = '(CNN)  -- Three scientists won the Nobel Prize in physics on Tuesday for two breakthroughs that led to two major underpinnings of the digital age -- fiber optics and digital photography, the Royal Swedish Academy of Sciences said. Willard Boyle, left, and George Smith handle a charge-coupled device in 1974. Charles K. Kao, a British and U.S. citizen, won for "groundbreaking achievements concerning the transmission of light in fibers for optical communication." Willard S. Boyle, a Canadian and U.S. citizen, and George E. Smith, a U.S. citizen, "invented the first successful imaging technology using a digital sensor, a CCD (Charge-Coupled Device)." Kao in 1966 "made a discovery that led to a breakthrough in fiber optics. He carefully calculated how to transmit light over long distances via optical glass fibers," the academy said in a press release. Today, "optical fibers make up the circulatory system that nourishes our communication society" and "facilitate broadband communication such as the Internet," the academy said. Boyle and Smith\'s Charge-Coupled Device -- invented in 1969 -- "is the digital camera\'s electronic eye" and paved the way for digital photography. "It revolutionized photography, as light could now be captured electronically instead of on film. The digital form facilitates the processing and distribution of these images. CCD technology is also used in many medical applications, e.g. imaging the inside of the human body, both for diagnostics and for microsurgery." The Nobel Prizes are being awarded this week and next. The medicine award was handed out on Monday. The prizes for chemistry and literature will be awarded Wednesday and Thursday. The Nobel Peace Prize winner will be named on Friday, and the award in economics will be issued on Monday.'

In [51]:
inputs=tokenizer(sample_text,max_length=512, truncation=True, return_tensors='pt')

In [52]:
# summary_ids = model.generate(inputs['input_ids'], length_penalty=2.0, max_length=150, early_stopping=True)

summary_ids = model.generate(
    **inputs,
    max_length=150,
    temperature=0.9,  # Encourages more randomness
    top_k=50,         # Considers top 50 probable tokens
    top_p=0.9,        # Samples from the top 90% probability mass
    repetition_penalty=1.2,  # Penalizes token repetition
    early_stopping=True
)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [53]:
summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)

In [54]:
print(summary)

Three scientists won the Nobel Prize in physics. The breakthroughs led to two major underpinnings of the digital age. The prize is being awarded this week and next.


In [55]:
highlight = 'Breakthroughs led to fiber optics, digital photography . Optical fibers facilitate broadband communication such as the Internet . Scientists made it possible to capture light electronically instead of on film . Nobel Prizes handed out this week .'

In [ ]:
sample_text

'(CNN)Never mind cats having nine lives. A stray pooch in Washington State has used up at least three of her own after being hit by a car, apparently whacked on the head with a hammer in a misguided mercy killing and then buried in a field -- only to survive. That\'s according to Washington State University, where the dog -- a friendly white-and-black bully breed mix now named Theia -- has been receiving care at the Veterinary Teaching Hospital. Four days after her apparent death, the dog managed to stagger to a nearby farm, dirt-covered and emaciated, where she was found by a worker who took her to a vet for help. She was taken in by Moses Lake, Washington, resident Sara Mellado. "Considering everything that she\'s been through, she\'s incredibly gentle and loving," Mellado said, according to WSU News. "She\'s a true miracle dog and she deserves a good life." Theia is only one year old but the dog\'s brush with death did not leave her unscathed. She suffered a dislocated jaw, leg inju

In [ ]:
summary1='Nobel Prize in physics on Tuesday for two breakthroughs. Three scientists won the Nobel Prize in physics for two breakthroughs. The breakthroughs led to two major underpinnings of the digital age -- fiber optics and digital photography. The Nobel Prizes are being awarded this week and next.'

In [56]:
!pip install rouge

In [57]:
from rouge import Rouge

In [58]:
rouge = Rouge()


In [59]:
r2=rouge.get_scores(summary,highlight)

In [60]:
r2

[{'rouge-1': {'r': 0.24242424242424243,
   'p': 0.3076923076923077,
   'f': 0.2711864357483482},
  'rouge-2': {'r': 0.06060606060606061,
   'p': 0.07407407407407407,
   'f': 0.06666666171666703},
  'rouge-l': {'r': 0.24242424242424243,
   'p': 0.3076923076923077,
   'f': 0.2711864357483482}}]

In [ ]:
r1=rouge.get_scores(summary,test_df.iloc[0]['highlights'])

In [ ]:
r1

[{'rouge-1': {'r': 0.1, 'p': 0.09090909090909091, 'f': 0.09523809024943337},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.06666666666666667,
   'p': 0.06060606060606061,
   'f': 0.06349205850340175}}]

In [61]:
b_text='A transformer model is a neural network that learns context and thus meaning by tracking relationships in sequential data like the words in this sentence. Transformer models apply an evolving set of mathematical techniques, called attention or self-attention, to detect subtle ways even distant data elements in a series influence and depend on each other. First described in a 2017 paper from Google, transformers are among the newest and one of the most powerful classes of models invented to date. They’re driving a wave of advances in machine learning some have dubbed transformer AI.Stanford researchers called transformers “foundation models” in an August 2021 paper because they see them driving a paradigm shift in AI. The “sheer scale and scope of foundation models over the last few years have stretched our imagination of what is possible,” they wrote.'

In [62]:
b_high='Transformer models are powerful neural networks that use attention mechanisms to understand relationships in sequential data, enabling them to learn context and meaning effectively. Introduced in 2017 by Google, these models have driven significant advances in AI and are now considered foundational to the field, powering transformative innovations across various domains.'

In [63]:
b_gen="transformer models apply an evolving set of mathematical techniques to detect subtle ways distant data elements in a series influence and depend on each other. they're driving a wave of advances in machine learning some have dubbed'transformer ai'in an august 2021 paper, stanford researchers called transformers'foundation models '"

In [64]:
rouge.get_scores(b_gen,b_high)

[{'rouge-1': {'r': 0.13636363636363635,
   'p': 0.13953488372093023,
   'f': 0.13793102948341937},
  'rouge-2': {'r': 0.02, 'p': 0.02127659574468085, 'f': 0.02061855170581479},
  'rouge-l': {'r': 0.11363636363636363,
   'p': 0.11627906976744186,
   'f': 0.11494252373629299}}]

In [65]:
b2h=" Ana Ivanovic penalized a game after needing an emergency toilet break in Austria . Serbian blames her pre-match snack of yyogurtafter leaving the court against rules . Seventh seed recovers to reach quarterfinals, where she will play Julia Gorges of Germany . Israeli third seed Shahar Peer reaches the last eight at the HP Japan Women's Open ."
b2g="The World No. 1 Ana Ivanovic reached the quarterfinals of the Generali Ladies Linz tournament. She was penalized for taking too long in the toilet when she left the court. The Swiss left-hander beat Austria's Aiko Nakamura 6-2 6-0. Third seed Shahar Peer will next play Julia Gorges in the HP Japan Women's Open."

In [66]:
rouge.get_scores(b2g,b2h)

[{'rouge-1': {'r': 0.4, 'p': 0.4166666666666667, 'f': 0.408163260308205},
  'rouge-2': {'r': 0.18867924528301888,
   'p': 0.19230769230769232,
   'f': 0.19047618547664413},
  'rouge-l': {'r': 0.38, 'p': 0.3958333333333333, 'f': 0.38775509704289884}}]